# Data Conversion

This notebook transforms our soundfiles into a numerical dataset for our models.

In [1]:
#Potentially needed installations
#!pip install librosa
# !pip install playsound
# !pip install tqdm

In [2]:
#Imports
import numpy as np
import pandas as pd
import librosa
#from playsound import playsound
#This is used to show a progress bar in the terminal. Helpful as the conversion can take a while.
from tqdm import tqdm
import concurrent.futures

## Preparation of the Overview Dataframe

In [3]:
def prepare_overview(list_of_files):
    """
    This function takes a list of files and creates a joint dataframe for audio file conversion
    """
    overview = None
    for file_path in list_of_files:
        with open(file_path, "r") as file:
            if overview is None:
                overview = pd.read_csv(file, sep="\t")
            else:
                overview = pd.concat([overview, pd.read_csv(file, sep="\t")], ignore_index=True)
    return overview

In [4]:
def preprocess_overview(overview, folder_path):
    """
    This function takes a dataframe and preprocesses it for audio file conversion
    """
    # we only need files with a gender label
    overview=overview.dropna(subset=["gender"])
    #irrelevant columns for our analysis
    overview=overview.drop(columns=["variant", "segment", "sentence_id", "client_id", "up_votes", "down_votes", "locale"])
    #dropping all files that are not simply male or female
    overview=overview[(overview["gender"]=="female_feminine")|(overview["gender"]=="male_masculine")]
    #changing the path to reflect the location of the audio files
    overview["path"]=overview["path"].apply(lambda x: f"{folder_path}/{x}")
    return overview.reset_index(drop=True)

In [5]:
list_of_files = ["other_18.tsv", "other_19.tsv", "other_20.tsv", "other_21.tsv", "validated_18.tsv", "validated_19.tsv", "validated_20.tsv", "validated_21.tsv"]
overview=prepare_overview(list_of_files)
overview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94691 entries, 0 to 94690
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   client_id        94691 non-null  object 
 1   path             94691 non-null  object 
 2   sentence_id      94691 non-null  object 
 3   sentence         94691 non-null  object 
 4   sentence_domain  727 non-null    object 
 5   up_votes         94691 non-null  int64  
 6   down_votes       94691 non-null  int64  
 7   age              68529 non-null  object 
 8   gender           46549 non-null  object 
 9   accents          58440 non-null  object 
 10  variant          0 non-null      float64
 11  locale           94691 non-null  object 
 12  segment          0 non-null      float64
dtypes: float64(2), int64(2), object(9)
memory usage: 9.4+ MB


In [6]:
# On my device, all audio files are in the same "clips" folder. Please change the path if your files are in a different location.
overview=preprocess_overview(overview, "./clips")
overview.head()

,path,sentence,sentence_domain,age,gender,accents
0,./clips/common_voice_en_40187693.mp3,Cohousing cultivates a culture of sharing and ...,NaN,fifties,male_masculine,Canadian English
1,./clips/common_voice_en_40187694.mp3,She is allegedly an agent of an organization c...,NaN,fifties,male_masculine,Canadian English
2,./clips/common_voice_en_40187695.mp3,Artume appeared as a recurring character in Ma...,NaN,fifties,male_masculine,Canadian English
3,./clips/common_voice_en_40187696.mp3,In practice however the resistive element vari...,NaN,fifties,male_masculine,Canadian English
4,./clips/common_voice_en_40187697.mp3,He then led the life of a wandering hermit.,NaN,fifties,male_masculine,Canadian English


In [7]:
overview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45358 entries, 0 to 45357
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   path             45358 non-null  object
 1   sentence         45358 non-null  object
 2   sentence_domain  384 non-null    object
 3   age              44929 non-null  object
 4   gender           45358 non-null  object
 5   accents          38582 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [8]:
overview.describe(include="all")

,path,sentence,sentence_domain,age,gender,accents
count,45358,45358,384,44929,45358,38582
unique,45358,45272,30,7,2,44
top,./clips/common_voice_en_42221538.mp3,The test is unofficial and just what it will p...,general,fourties,female_feminine,Scottish English
freq,1,3,241,15300,28975,14275


In [9]:
overview.groupby("gender").size()

gender
female_feminine    28975
male_masculine     16383
dtype: int64

We have a slight class imbalance that we may want to remove/have to deal with within the model building.

## Conversion of the Audiofiles

In [10]:
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True)

        # Feature extraction
        # MFCC (Mel-frequency cepstral coefficients) gives the timbre of the audio
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        
        # Chroma feature gives the harmonic content of the audio
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        
        # Spectral features
        # Spectral centroid gives the center of mass of the spectrum
        spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        # Spectral bandwidth gives the width of the spectrum
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        # Spectral contrast gives the difference in amplitude between peaks and valleys in the sound spectrum
        spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        # Spectral rolloff gives the frequency below which a certain percentage of the total spectral energy is contained
        spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        
        # Zero crossing rate gives the rate at which the signal changes sign
        zcr = librosa.feature.zero_crossing_rate(y)
        # Root mean square energy gives the energy of the signal (i.e., the loudness)
        rmse = librosa.feature.rms(y=y)

        # Combine features into a single feature vector
        features = np.hstack([
            np.mean(mfccs, axis=1), np.std(mfccs, axis=1),
            np.mean(chroma, axis=1), np.std(chroma, axis=1),
            np.mean(spec_centroid), np.std(spec_centroid),
            np.mean(spec_bw), np.std(spec_bw),
            np.mean(spec_contrast, axis=1), np.std(spec_contrast, axis=1),
            np.mean(spec_rolloff), np.std(spec_rolloff),
            np.mean(zcr), np.std(zcr),
            np.mean(rmse), np.std(rmse)
        ])

        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [11]:
def build_columns():
    columns = []
    columns += [f"mfcc_{i+1:02d}_mean" for i in range(20)]
    columns += [f"mfcc_{i+1:02d}_std" for i in range(20)]
    columns += [f"chroma_{i+1:02d}_mean" for i in range(12)]
    columns += [f"chroma_{i+1:02d}_std" for i in range(12)]
    columns += ["spec_centroid_mean", "spec_centroid_std"]
    columns += ["spec_bandwidth_mean", "spec_bandwidth_std"]
    columns += [f"spec_contrast_band_{i+1}_mean" for i in range(7)]
    columns += [f"spec_contrast_band_{i+1}_std" for i in range(7)]
    columns += ["spec_rolloff_mean", "spec_rolloff_std"]
    columns += ["zcr_mean", "zcr_std"]
    columns += ["rmse_mean", "rmse_std"]
    return columns

In [12]:
def extract_features_from_dataframe(df_paths, output_csv_path, parallel=True):
    features_list = []
    valid_indices = []

    paths = df_paths['path'].tolist()

    # Use ThreadPoolExecutor for parallel processing
    # This helps to speed up the feature extraction process
    if parallel:
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            futures = {executor.submit(extract_features, path): idx for idx, path in enumerate(paths)}
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(paths), desc="Extracting features"):
                result = future.result()
                if result is not None:
                    features_list.append(result)
                    valid_indices.append(futures[future])
    else:
        for idx, path in tqdm(enumerate(paths), total=len(paths), desc="Extracting features"):
            result = extract_features(path)
            if result is not None:
                features_list.append(result)
                valid_indices.append(idx)

    # Build features DataFrame
    columns = build_columns()
    features_df = pd.DataFrame(features_list, columns=columns)
    
    # Match features to original DataFrame
    merged_df = df_paths.iloc[valid_indices].reset_index(drop=True)
    final_df = pd.concat([merged_df, features_df], axis=1)

    # Save to CSV
    final_df.to_csv(output_csv_path, index=False)
    print(f"Saved extracted features to {output_csv_path}")

    return final_df

In [ ]:
df=extract_features_from_dataframe(overview, "./data/data.csv", parallel=True)
df.describe()

Extracting features: 100%|██████████| 45358/45358 [40:20<00:00, 18.74it/s]  


Saved extracted features to data.csv


,mfcc_01_mean,mfcc_02_mean,mfcc_03_mean,mfcc_04_mean,mfcc_05_mean,mfcc_06_mean,mfcc_07_mean,mfcc_08_mean,mfcc_09_mean,mfcc_10_mean,...,spec_contrast_band_4_std,spec_contrast_band_5_std,spec_contrast_band_6_std,spec_contrast_band_7_std,spec_rolloff_mean,spec_rolloff_std,zcr_mean,zcr_std,rmse_mean,rmse_std
count,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,...,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000,45358.000000
mean,-413.312954,93.823479,-11.062694,19.232826,0.018813,3.887656,-10.201332,-5.936500,-11.200684,-3.908554,...,5.023911,5.531940,8.393671,6.018052,4630.727460,2636.063587,0.113118,0.089776,0.044959,0.050718
std,77.021224,22.966793,19.415232,13.728738,13.573066,12.351489,7.570344,9.244700,8.631111,6.384909,...,0.707053,1.086150,2.189325,2.898502,1161.856951,549.878533,0.037467,0.023280,0.028279,0.026065
min,-1009.371521,8.527754,-109.845596,-29.976082,-99.547256,-42.214890,-60.356888,-39.346924,-43.236217,-32.694038,...,1.088558,1.785507,1.225389,0.830508,614.009534,132.853695,0.014346,0.010137,0.000007,0.000007
25%,-437.399200,76.560682,-25.662302,8.164198,-7.747386,-5.363337,-14.276426,-11.813797,-18.112986,-7.261864,...,4.606838,4.893319,6.935425,4.516315,3820.282388,2391.773151,0.087264,0.076256,0.031706,0.038324
50%,-410.583328,91.422825,-14.327928,18.614068,-1.315416,0.800746,-9.806545,-7.440382,-12.065773,-3.358245,...,5.019537,5.593451,8.515515,5.110147,4653.010912,2770.039084,0.111559,0.089141,0.039839,0.047953
75%,-377.211464,110.986599,1.453691,28.406997,9.326246,13.201524,-5.679545,-0.594520,-5.213984,-0.340330,...,5.397709,6.160684,9.897321,6.460226,5395.250006,2985.874105,0.133434,0.103205,0.051988,0.058496
max,-166.984146,193.673172,64.433517,105.173409,63.279827,62.071381,19.852123,46.526283,24.392496,39.869076,...,9.229922,11.747129,22.740114,22.756141,9875.266638,4217.979050,0.388499,0.226764,0.357266,0.271961
